챗봇에 메모리를 추가하지 않으면 챗봇은 아무것도 기억할 수 없다.

오픈 AI에서 제공하는 기본 API는 랭체인 없이 사용할 수 있지만 메모리를 지원하지 않자만, chatGPT에는 메모리가 탑재되어 있기 때문에 사용자가 이용할 떄 실제 사람과 대화를 예기하고 있다는 느낌을 들게 한다.


#### 1. Conversation Buffer Memory

단순히 이전 대화 내용 전체를 저장하는 메모리이지만 이 메모리의 단점은 대화 내용이 길어질수록 메모리도 계속 커지니까 비효울적이다.


#### 2. Conversation Buffer Window

시간에 따른 대화의 상호 작용 목록을 유지한다. 마지막 K개의 상호 작용만 사용한다. Conversation Buffer Memory의 단점을 보완해주지지만, 챗봇이 예전 대화를 기억하기 힘들며 최근에 일어난 대화에만 집중한다.


In [14]:
from langchain.memory import ConversationBufferWindowMemory


memory = ConversationBufferWindowMemory(
    return_messages=True,
    k=4
)

def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})

In [15]:
add_message(1, 1)

In [16]:
add_message(2, 2)
add_message(3, 3)
add_message(4, 4)

In [17]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='1'),
  AIMessage(content='1'),
  HumanMessage(content='2'),
  AIMessage(content='2'),
  HumanMessage(content='3'),
  AIMessage(content='3'),
  HumanMessage(content='4'),
  AIMessage(content='4')]}

In [18]:
add_message(5, 5)

In [19]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='2'),
  AIMessage(content='2'),
  HumanMessage(content='3'),
  AIMessage(content='3'),
  HumanMessage(content='4'),
  AIMessage(content='4'),
  HumanMessage(content='5'),
  AIMessage(content='5')]}